In [8]:
import pandas as pd
import numpy as np
import nltk
import string
from tqdm import tqdm



from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.experimental import enable_halving_search_cv  # noqa
from sklearn.model_selection import HalvingRandomSearchCV
from sklearn.model_selection import train_test_split

In [9]:
tqdm.pandas()

In [10]:
input_path = 'data/input/'

In [12]:
train = pd.read_csv(input_path + 'train_data_prepped.csv').fillna('')
test = pd.read_csv(input_path + 'test_data_prepped.csv').fillna('')

In [13]:
train[train.isnull().any(axis=1)]

,id,text,cat,text_no_punc,text_no_numerals,text_no_sw,text_porter_stemmed,text_lancaster_stemmed,text_lemmatized


In [14]:
test[test.isnull().any(axis=1)]

,id,text,cat,text_no_punc,text_no_numerals,text_no_sw,text_porter_stemmed,text_lancaster_stemmed,text_lemmatized


load train targets, binarize, drop nuetrals

In [16]:
train_results = pd.read_csv(input_path + 'train_results.csv')
train_results['target'].value_counts()

positive    520436
negative    519803
neutral         84
Name: target, dtype: int64

In [21]:
train['target'] = train_results['target']

drop nuetral target rows to keep it binary

In [23]:
train = train[train['target'] != 'neutral']
train['target'] = [1 if t=='positive' else 0 for t in train['target'].values]
train['target'].value_counts()


1    520436
0    519803
Name: target, dtype: int64

### prepare bag of words<br>
we need a different model for each text format

Naive Bayes doesn't have any hyperparameters, so we'll test different versions of the text processing with different versions of naive bayes and corpos generation

the following parameters will be tested<br>
1. size of corpus vocab
2. corpus ngram varieties
3. bernoulli, multinomial or gaussian naive bayes
4. stemming/lemmatizing methods

In [25]:
vocab_sizes = [100, 1_000, 10_000, None]
ngrams = [(1, 1), (2, 2), (1, 2), (1, 3)]
models = [BernoulliNB(), GaussianNB(), MultinomialNB()]
text_columns = ['text_no_numerals', 'text_no_sw', 'text_porter_stemmed', 'text_lancaster_stemmed', 'text_lemmatized']
threshold = 0.5

# shuffle train data
train = train.sample(len(train))

# get data and target
train_X = train.iloc[:, :-1]
train_y  = train.iloc[:, -1]

# get 5000 data points for first round of training
train_v1 = train_X[:5000]
targets_v1 = train_y[:5000]

In [29]:
targets_v1.head()

51538     1
43617     0
327072    0
304537    1
549680    1
Name: target, dtype: int64

In [26]:
def trainModels(df, targets, vocab_sizes, models, text_columns, ngrams, threshold):

    param_tracker = []
    counter = 1
    for size in tqdm(vocab_sizes):
#         print(f'vocab size: {size}')

        for model in tqdm(models):
#             print(f'model type: {model}')

            for col in tqdm(text_columns):
#                 print(f'processing method: {col}')


                for ng in tqdm(ngrams):
#                     print(f'ngram range: {ng}')

                    counter+=1
                    if counter % 10==0:
                        print(f'{counter}')
                    vectorizer = CountVectorizer(max_features=size, ngram_range=ng)

#                     print('creating bag of words')
                    BOW = vectorizer.fit_transform(df[col])
                    BOW_array = BOW.toarray()

                    X_train, X_valid, y_train, y_valid = train_test_split(BOW_array, targets, test_size=0.2, random_state=42)
#                     print('fitting data')      
                    model.fit(X_train, y_train)


                    prob_predictions = model.predict_proba(X_valid)

                    # gets index of 1 column in prob_predictions
                    pos_predictions = [pred[list(model.classes_).index(1)] for pred in prob_predictions]

                    num_correct_pred = 0
                    for pred, actual in zip(pos_predictions, y_valid):
                        if pred >= threshold:
                            binary_pred = 1
                        else:
                            binary_pred = 0

                        if binary_pred==actual:
                            num_correct_pred+=1
                    accuracy = num_correct_pred / len(y_valid)
#                     print(f'accuracy: {accuracy}')
#                     print('\n\n')

                    param_tracker.append((size, str(model), col, ng, accuracy))
                    
    param_df = pd.DataFrame(param_tracker, columns = ['vocab_size', 'model_type', 'text_column', 'ngram', 'accuracy'])
    param_df = param_df.sort_values(by='accuracy', ascending=False)
    return param_df

                    
                
    # for vocab size we'll test

In [27]:
param_df = trainModels(train_v1, targets_v1, vocab_sizes, models, text_columns, ngrams, threshold)

  0%|                                                                                             | 0/3 [00:00<?, ?it/s]

  0%|                                                                                             | 0/5 [00:00<?, ?it/s]


  0%|                                                                                             | 0/4 [00:00<?, ?it/s]


 25%|█████████████████████▎                                                               | 1/4 [00:00<00:00,  7.28it/s]


 50%|██████████████████████████████████████████▌                                          | 2/4 [00:00<00:00,  6.95it/s]


 75%|███████████████████████████████████████████████████████████████▊                     | 3/4 [00:00<00:00,  6.32it/s]


100%|█████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00,  5.29it/s]


 20%|█████████████████                                                                    | 1/5 [00:00<00:03,  1.32it/s]


  0%|            

10





100%|█████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00,  7.78it/s]


 60%|███████████████████████████████████████████████████                                  | 3/5 [00:01<00:01,  1.73it/s]


  0%|                                                                                             | 0/4 [00:00<?, ?it/s]


 50%|██████████████████████████████████████████▌                                          | 2/4 [00:00<00:00, 13.21it/s]


100%|█████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00,  8.69it/s]


 80%|████████████████████████████████████████████████████████████████████                 | 4/5 [00:02<00:00,  1.87it/s]


  0%|                                                                                             | 0/4 [00:00<?, ?it/s]


 50%|██████████████████████████████████████████▌                                          | 2/4 [00:00<00:00, 11.42it/s]

20





100%|█████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00,  7.40it/s]


100%|█████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:02<00:00,  1.77it/s]

 33%|████████████████████████████▎                                                        | 1/3 [00:02<00:05,  2.83s/it]

  0%|                                                                                             | 0/5 [00:00<?, ?it/s]


  0%|                                                                                             | 0/4 [00:00<?, ?it/s]


 50%|██████████████████████████████████████████▌                                          | 2/4 [00:00<00:00,  9.08it/s]


 75%|███████████████████████████████████████████████████████████████▊                     | 3/4 [00:00<00:00,  6.91it/s]


100%|█████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00,  5.36it/s]


 20%|██████████

...

170





 25%|█████████████████████▎                                                               | 1/4 [00:01<00:03,  1.05s/it]


 50%|██████████████████████████████████████████▌                                          | 2/4 [00:02<00:02,  1.11s/it]


 75%|███████████████████████████████████████████████████████████████▊                     | 3/4 [00:03<00:01,  1.18s/it]


100%|█████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:04<00:00,  1.17s/it]


 60%|███████████████████████████████████████████████████                                  | 3/5 [00:14<00:09,  4.72s/it]


  0%|                                                                                             | 0/4 [00:00<?, ?it/s]


 25%|█████████████████████▎                                                               | 1/4 [00:01<00:03,  1.05s/it]


 50%|██████████████████████████████████████████▌                                          | 2/4 [00:02<00:02,  1.19s/it]


 75%|████████

180





 75%|███████████████████████████████████████████████████████████████▊                     | 3/4 [00:03<00:01,  1.17s/it]


100%|█████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:04<00:00,  1.19s/it]


100%|█████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:23<00:00,  4.77s/it]

  0%|                                                                                             | 0/3 [00:00<?, ?it/s]

  0%|                                                                                             | 0/5 [00:00<?, ?it/s]


  0%|                                                                                             | 0/4 [00:00<?, ?it/s]


 25%|█████████████████████▎                                                               | 1/4 [00:02<00:06,  2.02s/it]


 50%|██████████████████████████████████████████▌                                          | 2/4 [00:09<00:10,  5.02s/it]


 75%|██████████

KeyboardInterrupt: 

In [ ]:
param_df[:20]

In [ ]:
# param_df.to_csv('data/output/naive_bayes_params.csv', index=False)

In [30]:
param_df = pd.read_csv('data/output/naive_bayes_params.csv')

# ngrams get saved as string, convert back to int tuple
param_df['ngram'] = [(int(ng[1]), int(ng[4])) for ng in param_df['ngram'].values]

based on these results, we'll train the full dataset on these 20 combinations

In [32]:
top_params = param_df[:10]

In [36]:
num_points = 100_000
num_train = int(num_points*0.8)

In [37]:
len(train)

1040239

In [ ]:

full_data_accuracy = []
for i, param_set in enumerate(tqdm(top_20.values)):
    vocab_size, model_type, text_column, ngram, _ = param_set
    print(vocab_size, model_type, text_column, ngram,)
    
    print('splitting train-validation data')
    
    # train is all training data with targets included!
    train = train.sample(len(train))
    training_data = train_X.iloc[:num_points]
    targets  = train_y.iloc[:num_points]
    
    
    

   
    
#     print(len(X_train), len(y_train), len(X_valid), len(y_valid))
    
    if vocab_size:
        vocab_size = int(vocab_size)
    
    if model_type == 'MultinomialNB()':
        model = MultinomialNB()
    elif model_type == 'BernoulliNB()':
        model = BernoulliNB()
    elif model_type == 'GaussianNB()':
        model = GaussianNB()
    else:
        print('warning, unrecognized model!')
        print(i, param_set)
        
    print('vectorizing')
    vectorizer = CountVectorizer(max_features=vocab_size, ngram_range=ngram)

    print('creating BOW')
    
    BOW = vectorizer.fit_transform(training_data[text_column])
    BOW_array = BOW.toarray()
    
    X_train = BOW_array[:num_train]
    y_train = training_targets[:num_train]
    
    X_valid = BOW_array[num_train:]
    y_valid = training_targets[num_train:]
    
    
    
    #     X_train, X_valid, y_train, y_valid = train_test_split(BOW_array, train_y, test_size=0.2, random_state=42)
    
    print('fitting model')
    model.fit(X_train, y_train)

    print('getting predictions')
    prob_predictions = model.predict_proba(X_valid)

    # gets index of 1 column in prob_predictions
    pos_predictions = [pred[list(model.classes_).index(1)] for pred in prob_predictions]
    
    
    # get accuracy
    num_correct_pred = 0
    for pred, actual in zip(pos_predictions, y_valid):
        if pred >= threshold:
            binary_pred = 1
        else:
            binary_pred = 0

        if binary_pred==actual:
            num_correct_pred+=1
    accuracy = num_correct_pred / len(y_valid)
    print(accuracy)
    print('\n')
    
    full_data_accuracy.append(accuracy)
    
    

In [ ]:
top_20['final_validation_acc'] = full_data_accuracy

In [ ]:
top_20.to_csv('top_20_naive_bayes.csv', index=False)

## train final model with best parameters

In [ ]:
#10000.0 BernoulliNB() text_lemmatized (1, 2)

print('vectorizing')
vectorizer = CountVectorizer(max_features=10000, ngram_range=(1, 2))

print('creating BOW')

BOW = vectorizer.fit_transform(training_data['text_lemmatized'])
BOW_array = BOW.toarray()


model = BernoulliNB()
model.fit()

In [ ]:
len(train_y)

In [ ]:
train_y